In [8]:
import sqlite3
import pickle
import pandas as pd

In [9]:
shutuba_table = pd.read_pickle("syutuba_table_test.pickle")
shutuba_table.drop(columns=['斤量','jockey_id', 'trainer_id', 'トレーナー名', '体重'\
                            , '体重変化', 'n_horses', 'course_len', 'weather' \
                            , 'race_type', 'ground_state', 'around'], inplace=True)

In [10]:
location = {'01' : '札幌',
             '02' : '函館',
             '03' : '福島',
             '04' : '新潟',
             '05' : '東京',
             '06' : '中山',
             '07' : '中京' ,
             '08' : '京都',
             '09' : '阪神',
             '10' : '小倉', }

In [11]:
shutuba_table = shutuba_table.astype(str) # 全体を文字列に
shutuba_table.reset_index(inplace=True)
shutuba_table = shutuba_table.rename(columns={'index': 'race_id'}) # インデックスを列に
shutuba_table['race_num'] = shutuba_table['race_id'].str[-2:] # race_num列を追加
shutuba_table['開催'] = shutuba_table['開催'].replace(location) # 開催地をIDから地名

In [12]:
print(shutuba_table)

         race_id 枠番  馬番    単勝    horse_id        馬名   騎手名  性 年齢        date  \
0   202302020511  1   1   6.5  2021100359  スカイキャンバス   横山武  牝  2  2023-07-15   
1   202302020511  2   2   5.8  2021106655   ロータスワンド   藤岡佑  牝  2  2023-07-15   
2   202302020511  2   3  45.4  2021103859    チークタイム   和田竜  牝  2  2023-07-15   
3   202302020511  3   4  54.7  2021100527   タヤスロンドン    丹内  牡  2  2023-07-15   
4   202302020511  3   5  20.2  2021101703  ナスティウェザー    富田  牡  2  2023-07-15   
5   202302020511  4   6   5.7  2021105668   バスターコール  ルメール  牡  2  2023-07-15   
6   202302020511  4   7  13.6  2021100356   クールベイビー   佐々木  牝  2  2023-07-15   
7   202302020511  5   8  75.5  2021103325   カレンナオトメ     黛  牝  2  2023-07-15   
8   202302020511  5   9  13.9  2021102392       ナナオ   西村淳  牝  2  2023-07-15   
9   202302020511  6  10  39.7  2021100653       アガシ    武豊  牡  2  2023-07-15   
10  202302020511  6  11  67.0  2021102254     コルルディ   鮫島駿  牝  2  2023-07-15   
11  202302020511  7  12  14.4  2021102775   レガテアドール 

In [29]:
conn = sqlite3.connect('races.sqlite')
cursor = conn.cursor()

******************

racesテーブル

In [15]:
race_table = shutuba_table.drop_duplicates(subset='race_id')[['race_id', 'date', 'race_name', 'race_num', '開催']]

In [16]:
race_table.rename(columns={'race_id' : 'id', 
                           'race_name' : 'name',
                           'race_num' : 'number',
                           '開催' : 'location'}, 
                  inplace=True)

In [17]:
print(race_table)

             id        date   name number location
0  202302020511  2023-07-15  函館2歳S     11       函館


In [18]:
race_table.to_sql('races', conn, if_exists='append', index=None)

1

*************

horsesテーブル

In [19]:
horse_table = shutuba_table.drop_duplicates(subset='horse_id')[['horse_id', '馬名', '年齢', '性']]

In [20]:
horse_table.rename(columns={'horse_id' : 'id', 
                           '馬名' : 'name',
                           '年齢' : 'age',
                           '性' : 'gender'}, 
                  inplace=True)

In [21]:
old = pd.read_sql_query("SELECT id FROM horses", conn)  # 登録済みのidを取得
mask = ~horse_table.iloc[:, 0].isin(old.iloc[:, 0])
new_horse_table = horse_table[mask]

In [22]:
new_horse_table.to_sql('horses', conn, if_exists='append', index=None)

15

**************

runnersテーブル

In [23]:
runner_table = shutuba_table[['race_id', '馬番', '枠番', '騎手名', '単勝', 'horse_id']].copy()

In [24]:
runner_table.rename(columns={'馬番' : 'number', 
                             '枠番' : 'position',
                             '騎手名' : 'jockey',
                             '単勝' : 'odds'}, 
                  inplace=True)

In [25]:
runner_table.to_sql('runners', conn, if_exists='append', index=None)

15

**************

In [32]:
# cursor.execute('SELECT * FROM races')
# data = cursor.fetchall()

# # データを表示
# for row in data:
#     print(row)

('202302020511', '2023-07-15', '函館2歳S', '11', '函館', None)


In [27]:
conn.close()